In [ ]:
!git clone https://github.com/AIstrophysics/chippr.git

In [ ]:
%cd chippr

In [ ]:
import sys
sys.path.append('/home/jovyan/AST424/chippr')

In [ ]:
%cd /home/jovyan/AST424/chippr

In [ ]:
pip install -r /home/jovyan/AST424/chippr/requirements.txt

In [ ]:
import sys
sys.path.append('/home/jovyan/AST424/chippr/chippr')
print(sys.path)

In [ ]:
import chippr
print(dir(chippr))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
import cProfile, pstats
from io import StringIO  #, StringIO in python2, changed for python3
import os

import chippr

In [ ]:
true_amps = np.array([0.20, 0.35, 0.55])
true_means = np.array([0.5, 0.2, 0.75])
true_sigmas = np.array([0.4, 0.2, 0.1])
n_mix_comps = len(true_amps)

true_funcs = []
for c in range(n_mix_comps):
    true_funcs.append(chippr.gauss(true_means[c], true_sigmas[c]**2))

true_nz = chippr.gmix(true_amps, true_funcs, limits=(0., 1.))

Input params.txt file, could be downloaded from Github: https://github.com/AIstrophysics/chippr/blob/master/docs/notebooks

In [ ]:
param_loc = '/home/jovyan/AST424/params.txt'
params = chippr.utils.ingest(param_loc)
params = chippr.defaults.check_sim_params(params) 
print(params)

Discrete distribution- uniform distribution is used here as the interim prior redshift distribution (serves as p(phi)). Check later if the continous distribution works here? 

In [ ]:
bin_ends = np.array([params['bin_min'], params['bin_max']])
weights = np.array([1.])

int_prior = chippr.discrete(bin_ends, weights)

In [ ]:
results_loc = os.path.join(os.path.join(os.path.join(os.path.join('..', '..'), 'research'), 'results'), 'demo')

posteriors = chippr.catalog(params=param_loc, loc=results_loc)

output = posteriors.create(true_nz, int_prior)

data = np.exp(output['log_interim_posteriors'])

In [ ]:
plt.hist(posteriors.samps.T[1], bins=100, density=True, color="k")
plt.plot(posteriors.z_coarse, true_nz.evaluate(posteriors.z_coarse), "r-")
#plt.plot(posteriors.z_coarse, np.sum(data, axis=0) / 10**params['n_gals'], "go")
plt.xlabel("z")